In [14]:
import h3
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from shapely.geometry import Polygon
import json

In [2]:
def basic_scatter_mapbox():
    # Sample data - cities with their coordinates
    df = pd.DataFrame({
        'city': ['New York', 'Los Angeles', 'Chicago', 'Houston'],
        'lat': [40.7128, 34.0522, 41.8781, 29.7604],
        'lon': [-74.0060, -118.2437, -87.6298, -95.3698],
        'population': [8400000, 4000000, 2700000, 2300000]
    })

    # Create scatter mapbox plot
    fig = px.scatter_mapbox(
        df,
        lat='lat',
        lon='lon',
        hover_name='city',
        size='population',
        size_max=30,
        zoom=3,
        title='Major US Cities'
    )

    # Use OpenStreetMap style (free, no token needed)
    fig.update_layout(
        mapbox_style='open-street-map',
        margin={"r":0,"t":30,"l":0,"b":0}
    )
    
    return fig

# Example 2: Choropleth Map
def us_state_choropleth():
    # Sample state-level data
    df_states = pd.DataFrame({
        'state_code': ['NY', 'CA', 'TX', 'FL'],
        'value': [200, 300, 250, 180]
    })

    # Create choropleth map
    fig = go.Figure(data=go.Choropleth(
        locations=df_states['state_code'],
        z=df_states['value'],
        locationmode='USA-states',
        colorscale='Viridis',
        colorbar_title="Value"
    ))

    fig.update_layout(
        title_text='US States Choropleth Map',
        geo_scope='usa'
    )
    
    return fig

In [3]:
fig_scatter = basic_scatter_mapbox()
fig_scatter.show()
    
    # Create and show the choropleth
fig_choropleth = us_state_choropleth()
fig_choropleth.show()

In [7]:
df_states = pd.DataFrame({
    'state_code': ['NY', 'CA', 'TX', 'FL', 'IL', 'PA', 'OH', 'GA', 'MI', 'NC'],
    'region': ['Northeast', 'West', 'South', 'South', 'Midwest', 
               'Northeast', 'Midwest', 'South', 'Midwest', 'South'],
    'risk_level': ['High', 'Medium', 'High', 'Low', 'Medium',
                   'Low', 'Medium', 'High', 'Low', 'Medium']
})
def categorical_choropleth_predefined():
    """
    Creates a choropleth map using predefined categories
    """
    # Define color mapping for regions
    color_map = {
        'Northeast': '#ff7f0e',  # Orange
        'West': '#1f77b4',      # Blue
        'South': '#2ca02c',     # Green
        'Midwest': '#d62728'    # Red
    }
    
    # Map categories to numerical values for plotting
    df_states['color_values'] = df_states['region'].map(
        {region: i for i, region in enumerate(color_map.keys())}
    )

    fig = go.Figure(data=go.Choropleth(
        locations=df_states['state_code'],
        z=df_states['color_values'],
        locationmode='USA-states',
        colorscale=[[i/len(color_map), color] for i, color in enumerate(color_map.values())],
        colorbar_ticktext=list(color_map.keys()),
        colorbar_tickvals=list(range(len(color_map))),
        showscale=True,
        colorbar_title="Region"
    ))

    fig.update_layout(
        title_text='US States by Region (Categorical)',
        geo_scope='usa',
    )
    
    return fig

def discrete_color_choropleth():
    """
    Creates a choropleth map with discrete colors for risk levels
    """
    # Define color scale for risk levels
    color_discrete_map = {
        'Low': '#2ecc71',    # Green
        'Medium': '#f1c40f', # Yellow
        'High': '#e74c3c'    # Red
    }
    
    # Create numerical mapping for risk levels
    risk_to_num = {'Low': 0, 'Medium': 1, 'High': 2}
    df_states['risk_num'] = df_states['risk_level'].map(risk_to_num)

    fig = go.Figure(data=go.Choropleth(
        locations=df_states['state_code'],
        z=df_states['risk_num'],
        locationmode='USA-states',
        colorscale=[[0, color_discrete_map['Low']], 
                   [0.5, color_discrete_map['Medium']], 
                   [1, color_discrete_map['High']]],
        colorbar_ticktext=['Low', 'Medium', 'High'],
        colorbar_tickvals=[0, 1, 2],
        showscale=True,
        colorbar_title="Risk Level"
    ))

    fig.update_layout(
        title_text='US States by Risk Level (Discrete)',
        geo_scope='usa',
    )
    
    return fig

def categorical_with_pattern():
    """
    Creates a choropleth map with both color and pattern fills
    """
    fig = go.Figure()

    # Add base choropleth
    fig.add_trace(go.Choropleth(
        locations=df_states['state_code'],
        z=df_states['risk_num'],
        locationmode='USA-states',
        colorscale=[[0, '#2ecc71'], [0.5, '#f1c40f'], [1, '#e74c3c']],
        showscale=True,
        colorbar_title="Risk Level",
        colorbar_ticktext=['Low', 'Medium', 'High'],
        colorbar_tickvals=[0, 1, 2],
        name='Risk Level'
    ))

    # Add text labels on states
    fig.add_trace(go.Scattergeo(
        locations=df_states['state_code'],
        locationmode='USA-states',
        text=df_states['region'],
        mode='text',
        textposition="middle center",
        name='Region'
    ))

    fig.update_layout(
        title_text='US States by Risk Level and Region',
        geo_scope='usa',
        showlegend=True
    )
    
    return fig


In [8]:
fig1 = categorical_choropleth_predefined()
fig1.show()
    
fig2 = discrete_color_choropleth()
fig2.show()
    
fig3 = categorical_with_pattern()
fig3.show()


In [40]:
import h3
import h3
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, box
import plotly.express as px



In [49]:
# Butantã approximate boundaries
BUTANTA_BOUNDS = {
    'min_lat': -23.5847,
    'max_lat': -23.5505,
    'min_lon': -46.7437,
    'max_lon': -46.7075
}

def get_h3_cells_in_area(bounds, resolution=10):
    """Get H3 cells within a bounding box."""
    # Create a rectangular boundary
    bbox = box(bounds['min_lon'], bounds['min_lat'], 
               bounds['max_lon'], bounds['max_lat'])
    
    # Get the center points in a grid
    lat_points = np.linspace(bounds['min_lat'], bounds['max_lat'], 20)
    lon_points = np.linspace(bounds['min_lon'], bounds['max_lon'], 20)
    
    # Get H3 cells for each point
    h3_cells = set()
    for lat in lat_points:
        for lon in lon_points:
            h3_cell = h3.latlng_to_cell(lat, lon, resolution)
            h3_cells.add(h3_cell)
    
    return list(h3_cells)

def h3_to_polygon(h3_address):
    """Convert H3 address to a Shapely polygon."""
    coords = h3.cell_to_boundary(h3_address)
    print(coords)
    print(coords[0])
    return Polygon(coords)

# Get H3 cells for Butantã
h3_cells = get_h3_cells_in_area(BUTANTA_BOUNDS)

# Create sample categories (you would replace this with real categories)
categories = ['Residential', 'Commercial', 'Mixed Use', 'Green Area', 'Educational']
np.random.seed(42)  # For reproducible results
sample_categories = np.random.choice(categories, size=len(h3_cells), p=[0.4, 0.2, 0.2, 0.1, 0.1])

# Create DataFrame
df = pd.DataFrame({
    'h3_index': h3_cells,
    'category': sample_categories
})

# Convert H3 indices to polygons
df['geometry'] = df['h3_index'].apply(h3_to_polygon)

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Create color map
color_map = {
    'Residential': '#FFB6C1',  # Light pink
    'Commercial': '#87CEEB',   # Sky blue
    'Mixed Use': '#DDA0DD',    # Plum
    'Green Area': '#90EE90',   # Light green
    'Educational': '#F0E68C'   # Khaki
}

# Plot using plotly
fig = px.choropleth_mapbox(gdf,
                          geojson=gdf.geometry.__geo_interface__,
                          locations=gdf.index,
                          color='category',
                          color_discrete_map=color_map,
                          title='Butantã Neighborhood - H3 Grid (Resolution 10)',
                          center={"lat": -23.5676, "lon": -46.7256},  # Butantã center
                          mapbox_style="carto-positron",
                          zoom=13,
                          height=800)

# Update layout
fig.update_layout(
    margin={"r":0,"t":30,"l":0,"b":0},
    mapbox=dict(
        style="carto-positron",
        zoom=13,
        center=dict(lat=-23.5676, lon=-46.7256)
    )
)

fig.show()

# Print some statistics
print(f"Total number of H3 cells: {len(h3_cells)}")
print("\nCategory distribution:")
print(df['category'].value_counts().to_frame())

((-23.576373565231968, -46.73958381349297), (-23.57596301973555, -46.738988793485724), (-23.57529934744696, -46.739114824205465), (-23.575046222039585, -46.73983587310807), (-23.57545676911816, -46.74043089139648), (-23.57612044002192, -46.740304862501155))
(-23.576373565231968, -46.73958381349297)
((-23.57151764053378, -46.707527966870394), (-23.57110692673697, -46.70693291957352), (-23.570443212840114, -46.70705899448685), (-23.570190214118703, -46.707780114875646), (-23.570600929495658, -46.708375160463376), (-23.571264642013848, -46.708249087371485))
(-23.57151764053378, -46.707527966870394)
((-23.585253888663157, -46.73722433072638), (-23.584843384092423, -46.73662929992253), (-23.584179780943852, -46.73675533575648), (-23.58392668375015, -46.737476400570046), (-23.584337189903152, -46.73807142965572), (-23.585000791667568, -46.73794539564604))
(-23.585253888663157, -46.73722433072638)
((-23.554354766112354, -46.708768572694815), (-23.553943958803877, -46.70817354292554), (-23.553

Total number of H3 cells: 400

Category distribution:
             count
category          
Residential    168
Mixed Use       84
Commercial      67
Green Area      43
Educational     38


In [50]:
gdf

,h3_index,category,geometry
0,8aa81005a907fff,Residential,"POLYGON ((-23.57637 -46.73958, -23.57596 -46.7..."
1,8aa8100ed797fff,Educational,"POLYGON ((-23.57152 -46.70753, -23.57111 -46.7..."
2,8aa81005e0e7fff,Mixed Use,"POLYGON ((-23.58525 -46.73722, -23.58484 -46.7..."
3,8aa8100e9467fff,Commercial,"POLYGON ((-23.55435 -46.70877, -23.55394 -46.7..."
4,8aa81005a0dffff,Residential,"POLYGON ((-23.56165 -46.73565, -23.56124 -46.7..."
...,...,...,...
395,8aa8100e97b7fff,Educational,"POLYGON ((-23.55113 -46.70736, -23.55072 -46.7..."
396,8aa8100ed72ffff,Green Area,"POLYGON ((-23.57322 -46.71326, -23.57281 -46.7..."
397,8aa81005e29ffff,Commercial,"POLYGON ((-23.57836 -46.73921, -23.57795 -46.7..."
398,8aa81005adb7fff,Mixed Use,"POLYGON ((-23.57480 -46.72642, -23.57439 -46.7..."


In [54]:
def plot_geodataframe(gdf):
    """
    Plot a GeoDataFrame using Plotly with different colors for each category
    
    Parameters:
    gdf (GeoDataFrame): Input geodataframe with 'category' and 'geometry' columns
    
    Returns:
    plotly.graph_objects.Figure
    """
    # Create a base figure
    fig = go.Figure()
    
    # Color mapping for different categories
    color_map = {
        'Residential': 'red',
        'Educational': 'blue',
        'Commercial': 'yellow',
        'Mixed Use': 'purple',
        'Green Area': 'green'
    }
    
    # Add traces for each category
    for category in gdf['category'].unique():
        category_data = gdf[gdf['category'] == category]
        
        for _, row in category_data.iterrows():
            # Extract coordinates from the geometry
            coords = row['geometry'].exterior.coords.xy
            
            # Add polygon to the map
            fig.add_trace(go.Scatter(
                x=tuple(coords[0]),
                y=tuple(coords[1]),
                fill='toself',
                fillcolor=color_map.get(category, 'gray'),
                line=dict(color='black', width=1),
                name=category,
                showlegend=True if coords[0][0] == category_data.iloc[0]['geometry'].exterior.coords.xy[0][0] else False,
                hovertemplate=f"Category: {category}<br>ID: {row['h3_index']}<extra></extra>"
            ))
    
    # Update layout
    fig.update_layout(
        title='Land Use Categories Map',
        showlegend=True,
        legend_title='Categories',
        hovermode='closest',
        yaxis=dict(
            scaleanchor='x',
            scaleratio=1
        )
    )
    
    # Update axes
    fig.update_xaxes(title='Longitude')
    fig.update_yaxes(title='Latitude')
    
    return fig

In [55]:

# Create and display the plot
fig = plot_geodataframe(gdf)
fig.show()